In [1]:
from ctu_crs import CRS97
import numpy as np
from numpy.typing import NDArray
import time
import json
import cv2
import PoseComposer
import camera
import puzzle

In [2]:
robot = CRS97()

Firmware version : MARS8 v1.0 build Pi Oct  2 2017 11:06:45


In [9]:
robot.initialize(home=False)

In [10]:
robot.soft_home()

In [17]:
robot.release()

In [11]:
def move_robot_to(position):
    q0 = robot.q_home
    ik_sols = robot.ik(position)
    assert len(ik_sols) > 0
    closest_solution = min(ik_sols, key=lambda q: np.linalg.norm(q - q0))
    robot.move_to_q(closest_solution)
    robot.wait_for_motion_stop()

In [12]:
T_RC_def = [[-4.91410691e-03, 9.98341087e-01, -5.73665839e-02, 4.90644721e+02],
                [9.97906036e-01, 1.19594619e-03, -6.46692634e-02, 2.38816422e+01],
                [-6.44933754e-02, -5.75642520e-02, -9.96256474e-01, 1.19549166e+03],
                [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]
T_RC_12rand = [[-3.09275095e-02 , 9.98889991e-01, -3.55285092e-02,  4.71849312e+02],
 [ 9.99249525e-01,  3.00701923e-02, -2.44165920e-02, -1.25522968e+01],
 [-2.33211402e-02 ,-3.62569903e-02 ,-9.99070345e-01 , 1.19959167e+03],
 [ 0.00000000e+00 , 0.00000000e+00 , 0.00000000e+00 , 1.00000000e+00]]

In [26]:
cam = camera.Camera()
img = robot.grab_image()
midpoint, R = cam.get_mid_points(img)
centerInRobotCoords = cam.cameraToRobot(midpoint)

[[2]
 [1]]


In [27]:
midpoint

array([ -21.82300377,   58.14589163, 1176.55287577])

In [28]:
midpoint[2] -= 35
midpoint

array([ -21.82300377,   58.14589163, 1141.55287577])

In [29]:
R

array([[ 0.9851665 ,  0.17160117,  0.        ],
       [-0.17160117,  0.9851665 ,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [30]:
centerInRobotCoords

array([ 0.48130657, -0.07391293,  0.02140355])

In [40]:
maze = puzzle.PuzzleA()
trajectory_R = maze.get_reverse_trajectory()
trajectory_F  = trajectory_R[::-1]
trajectory_F[:, 2] *= -1
trajectory_F

array([[   0.,    0., -200.],
       [   0.,    0., -180.],
       [   0.,    0., -160.],
       [   0.,    0., -140.],
       [   0.,    0., -120.],
       [   0.,    0., -100.],
       [   0.,    0.,  -80.],
       [   0.,    0.,  -60.],
       [   0.,    0.,  -40.],
       [   0.,    0.,  -20.]])

In [41]:
trajectory_F_camera_coord = trajectory_F @ R + midpoint
trajectory_F_camera_coord

array([[ -21.82300377,   58.14589163,  941.55287577],
       [ -21.82300377,   58.14589163,  961.55287577],
       [ -21.82300377,   58.14589163,  981.55287577],
       [ -21.82300377,   58.14589163, 1001.55287577],
       [ -21.82300377,   58.14589163, 1021.55287577],
       [ -21.82300377,   58.14589163, 1041.55287577],
       [ -21.82300377,   58.14589163, 1061.55287577],
       [ -21.82300377,   58.14589163, 1081.55287577],
       [ -21.82300377,   58.14589163, 1101.55287577],
       [ -21.82300377,   58.14589163, 1121.55287577]])

In [42]:
trajectory_F_robot_coord = [cam.cameraToRobot(x, T_RC=T_RC_12rand) for x in trajectory_F_camera_coord]    
# for p in trajectory_F_robot_coord:
#     p[2] += 0.05
trajectory_F_robot_coord

[array([ 0.49715362, -0.05559998,  0.25731486]),
 array([ 0.49644305, -0.05608831,  0.23733345]),
 array([ 0.49573248, -0.05657664,  0.21735204]),
 array([ 0.49502191, -0.05706497,  0.19737064]),
 array([ 0.49431134, -0.0575533 ,  0.17738923]),
 array([ 0.49360077, -0.05804164,  0.15740782]),
 array([ 0.4928902 , -0.05852997,  0.13742641]),
 array([ 0.49217963, -0.0590183 ,  0.11744501]),
 array([ 0.49146906, -0.05950663,  0.0974636 ]),
 array([ 0.49075849, -0.05999496,  0.07748219])]

In [43]:
start = trajectory_F_robot_coord[0].copy()
start[2] = start[2] + 0.1
start

array([ 0.49715362, -0.05559998,  0.35731486])

In [44]:
arm_vector = np.array([-1, 1, 0])
normal_vector = np.array([0, 0, 1])
StartPose = PoseComposer.make_se3_matrix(start, arm_vector, normal_vector)
StartPose

array([[-0.70710678,  0.70710678,  0.        ,  0.40169421],
       [ 0.70710678,  0.70710678,  0.        ,  0.03985944],
       [ 0.        ,  0.        , -1.        ,  0.35731486],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [45]:
move_robot_to(StartPose)

In [46]:
for position in trajectory_F_robot_coord:
    Pose = PoseComposer.make_se3_matrix(position, arm_vector, normal_vector)
    move_robot_to(Pose)

In [48]:
for position in trajectory_F_robot_coord[::-1]:
    Pose = PoseComposer.make_se3_matrix(position, arm_vector, normal_vector)
    move_robot_to(Pose)
move_robot_to(StartPose)

In [6]:
import solution

In [7]:
maze = puzzle.PuzzleA()
solution.solution(maze, robot, T_RC_12rand)

[[1]
 [2]]


In [44]:
robot.release()


In [43]:
robot.soft_home()

In [42]:
robot.initialize(home=False)